# Dataset-JSON Hackathon Notebook

This notebook was developed as part of the COSA Dataset-JSON Hackathon. It is experimental and meant as an exercise to explore working with Dataset-JSON in Python.

This notebook demonstrates using Python Pandas to convert Dataset-JSON files to dataframes that are then stored as a CSV file. Some examples that use streaming to iteratively process the Dataset-JSON files are included to show how files too big to fit into memory could be processed, though this has not been tested with a very large dataset. A Pandas dataframe is then converted back into Dataset-JSON to demonstrate creating a Dataset-JSON file from a dataframe.

### Import the Python libraries

In [19]:
import pandas as pd
import json
import ijson
import csv
from decimal import Decimal
import requests

### Get Dataset-JSON example file from GitHub

Retrieve an example Dataset-JSON file from the CDISC DataExchange-Dataset-Json repository and write the file in a local data directory.

In [21]:
data_file = 'data/vs.json'
try:
    r = requests.get('https://github.com/cdisc-org/DataExchange-DatasetJson/blob/master/examples/sdtm/vs.json?raw=True')
    r.raise_for_status()
except requests.exceptions.HTTPError as err:
    raise SystemExit(err)
with open(data_file, 'w') as f:
    json.dump(r.json(), f)

### Load Dataset-JSON using json module
For smaller datasets, simply load data using json module. This loads the entire file into memory so may not work for very large datasets.

In [22]:
with open(data_file, 'r') as f:
    data = json.loads(f.read())

Show the name and label for the dataset as well as all the variables names that will be used as column headings. Also, load the data types from the Dataset-JSON file to set the data types in the Pandas dataframe.

In [23]:
dataset_attrs = list(data["clinicalData"]["itemGroupData"].values())[0]
print(f"Name: {dataset_attrs['name']} ({dataset_attrs['label']})", end='\n\n')
variables = [var['name'] for var in dataset_attrs['items']]
print(f"Variables: {', '.join([var_name for var_name in variables])}")
data_types = [var['type'] for var in dataset_attrs['items']]

Name: VS (Vital Signs)

Variables: ITEMGROUPDATASEQ, STUDYID, DOMAIN, USUBJID, VSSEQ, VSTESTCD, VSTEST, VSPOS, VSORRES, VSORRESU, VSSTRESC, VSSTRESN, VSSTRESU, VSSTAT, VSLOC, VSLOBXFL, VSREPNUM, VISITNUM, VISIT, EPOCH, VSDTC, VSDY


### Create a dataframe from the Dataset-JSON file
Create a dataframe from the Dataset-JSON file. Then print the top 5 rows and provide the memory usage for the dataframe. Then save the dataframe as a CSV file.

In [24]:
df = pd.DataFrame(dataset_attrs['itemData'], columns=variables)
print(df.head(5), end='\n\n')
print(f"\ndataframe memory usage: {df.memory_usage().sum()} bytes")

   ITEMGROUPDATASEQ       STUDYID DOMAIN   USUBJID  VSSEQ VSTESTCD  \
0                 1  CDISCPILOT01     VS  CDISC001      1    DIABP   
1                 2  CDISCPILOT01     VS  CDISC001      2    DIABP   
2                 3  CDISCPILOT01     VS  CDISC001      3    DIABP   
3                 4  CDISCPILOT01     VS  CDISC001      4    DIABP   
4                 5  CDISCPILOT01     VS  CDISC001      5    DIABP   

                     VSTEST     VSPOS VSORRES VSORRESU  ... VSSTRESU  VSSTAT  \
0  Diastolic Blood Pressure  STANDING      71     mmHg  ...     mmHg           
1  Diastolic Blood Pressure  STANDING      71     mmHg  ...     mmHg           
2  Diastolic Blood Pressure  STANDING      83     mmHg  ...     mmHg           
3  Diastolic Blood Pressure  STANDING      79     mmHg  ...     mmHg           
4  Diastolic Blood Pressure  STANDING      68     mmHg  ...     mmHg           

  VSLOC VSLOBXFL VSREPNUM VISITNUM        VISIT      EPOCH       VSDTC VSDY  
0                   

Write the dataframe out as a CSV file.

In [25]:
df.to_csv('data/' + dataset_attrs['name'] + '.csv', index=False, encoding='utf-8')

### Create datatype conversion functions
Create helper functions that convert Dataset-JSON (and Define-XML) datatypes to pandas dataframe datatypes.

In [26]:
def decimal_from_value(value):
    """ helper function to convert a value to a Decimal datatype """
    return Decimal(value)


def convert_data_types(col_headers, define_data_types):
    """ helper method to lookup Dataset-JSON datatypes to Pandas datatypes """
    pandas_types = {}
    for idx, header in enumerate(col_headers):
        pandas_types[header] = lookup_pandas_data_type(define_data_types[idx])
    return pandas_types


def lookup_pandas_data_type(define_data_type):
    """ helper method to convert a given Dataset-JSON datatype to a Panda datatype """
    if define_data_type == "integer":
        return "int64"
    elif define_data_type == "string":
        return "object"
    elif define_data_type == "decimal":
        # store as object and convert on read to decimal
        return "object"
    elif define_data_type == "float" or define_data_type == "double":
        return "float64"
    elif define_data_type == "datetime":
        return "datetime64"
    else:
        print(f"unhandled Define-XML datatype: {define_data_type}. Assigning to object datatype.")
        return "object"


### Stream the Dataset-JSON file
Next we will stream, or iteratively parse, the Dataset-JSON file and write it out to CSV incrementally. Stream the Dataset-JSON file reading one record at a time (using ijson) and iteratively write a set of rows to the CSV file. Set missing VSREPNUM values to -99 to enable Pandas to set the datatype to integer.

In [27]:
# read Dataset-JSON data rows iteratively and write them to the csv file
DF_CHUNK_SIZE = 300
CSV_FILE = 'data/incremental_vs.csv'
def create_new_dataframe(rows):
     idf = pd.DataFrame(rows, columns=variables)
     idf.to_csv(CSV_FILE, index=True, encoding='utf-8', columns=variables)

def append_to_dataframe(rows):
     idf = pd.DataFrame(rows, columns=variables)
     idf.to_csv(CSV_FILE, index=True, encoding='utf-8', header=None, mode='a')

with open('data/vs.json', 'rb') as f:
    row_set = []
    rows = ijson.items(f, 'clinicalData.itemGroupData.IG.VS.itemData.item')
    for row_count, row in enumerate(rows):
        if row[16] is None:
            row[16] = -99
        row_set.append(row)
        if len(row_set) == DF_CHUNK_SIZE:
            idf = pd.DataFrame(row_set, columns=variables)
            if row_count > DF_CHUNK_SIZE:
                append_to_dataframe(row_set)
            else:
                create_new_dataframe(row_set)
            row_set = []
    if len(row_set) and row_count > DF_CHUNK_SIZE:
        append_to_dataframe(row_set)
    else:
        create_new_dataframe(row_set)
print(f"total incremental rows printed: {row_count + 1}", end='\n\n')

total incremental rows printed: 1414


### Create dataframes from the CSV file and show memory usage
Load the complete CSV file at once and convert the datatypes to pandas datatypes. Remove the datatype settings for the columns for which we have a converter. Print the memory usage for the entire dataset.

In [28]:
# load full csv with data types and date processing
pandas_dtypes = convert_data_types(variables, data_types)
# remove datatypes for which we will apply a converter
pandas_dtypes.pop('VSSTRESN', None)
pandas_dtypes.pop('VISITNUM', None)
csv_df = pd.read_csv(CSV_FILE,
                     index_col=0,
                     dtype=pandas_dtypes,
                     parse_dates=["VSDTC"],
                     converters={'VSSTRESN': decimal_from_value, 'VISITNUM': decimal_from_value},
                     verbose=True,
                     quoting=csv.QUOTE_NONE,
                     on_bad_lines='warn',
                     skip_blank_lines=False)
print(f"Rows in the incremental csv dataframe: {len(csv_df.axes[0])}")
print(f"full csv memory usage: {csv_df.memory_usage().sum()} bytes", end="\n\n")

Tokenization took: 1.48 ms
Type conversion took: 3.59 ms
Parser memory cleanup took: 0.01 ms
Rows in the incremental csv dataframe: 1414
full csv memory usage: 260176 bytes


To reduce the amount of memory used, this time load just 5 columns of the dataset. Show the top 5 records, the number of rows, and print the memory usage.

In [29]:
# load 5 columns only
cols = ['ITEMGROUPDATASEQ', 'USUBJID', 'VSTEST', 'VSORRES', 'VSORRESU']
col_dtypes = {col: pandas_dtypes[col] for col in cols}
csv_df = pd.read_csv('data/incremental_vs.csv',
                     usecols=cols,
                     dtype=col_dtypes,
                     on_bad_lines='warn',
                     verbose=True,
                     quoting=csv.QUOTE_NONE)
print(csv_df.head(n=5), end='\n\n')
print(f"5 column incremental dataframe row count: {len(csv_df.axes[0])}")
print(f"5 columns csv memory usage: {csv_df.memory_usage().sum()} bytes", end="\n\n")

Tokenization took: 1.34 ms
Type conversion took: 0.75 ms
Parser memory cleanup took: 0.00 ms
   ITEMGROUPDATASEQ   USUBJID                    VSTEST VSORRES VSORRESU
0                 1  CDISC001  Diastolic Blood Pressure      71     mmHg
1                 2  CDISC001  Diastolic Blood Pressure      71     mmHg
2                 3  CDISC001  Diastolic Blood Pressure      83     mmHg
3                 4  CDISC001  Diastolic Blood Pressure      79     mmHg
4                 5  CDISC001  Diastolic Blood Pressure      68     mmHg

5 column incremental dataframe row count: 1414
5 columns csv memory usage: 56688 bytes


Write the 5 column CSV to a compressed file.

In [30]:
csv_df.to_csv('data/vs_zip.csv.zip')

Read the compressed 5 columns dataset and print the memory usage. Note that compressing the dataset saves disk storage, but not the memory usage.

In [31]:
df_zip = pd.read_csv('data/vs_zip.csv.zip',
                     index_col=0,
                     dtype=pandas_dtypes,
                     converters={'VSSTRESN': decimal_from_value, 'VISITNUM': decimal_from_value})
print(df_zip.head(n=5), end='\n\n')
print(f"5 columns zipped csv memory usage: {df_zip.memory_usage().sum()} bytes", end="\n\n")

   ITEMGROUPDATASEQ   USUBJID                    VSTEST VSORRES VSORRESU
0                 1  CDISC001  Diastolic Blood Pressure      71     mmHg
1                 2  CDISC001  Diastolic Blood Pressure      71     mmHg
2                 3  CDISC001  Diastolic Blood Pressure      83     mmHg
3                 4  CDISC001  Diastolic Blood Pressure      79     mmHg
4                 5  CDISC001  Diastolic Blood Pressure      68     mmHg

5 columns zipped csv memory usage: 67872 bytes


### Convert the CSV file to Dataset-JSON
Next read in the 5 column VS dataset and write it out as Dataset-JSON. Start by creating a Dataset-JSON dictionary

In [14]:
vs_5col_json = {
  "clinicalData": {
    "studyOID": "hackathon.vs.reduced",
    "metaDataVersionOID": "MDV.MSGv2.0.SDTMIG.3.3.VS.5COL",
    "itemGroupData": {
      "IG.VS": {
        "records": len(csv_df.axes[0]),
        "name": "VS",
        "label": "Vital Signs",
        "items": [],
        "itemData": []
      }
    }
  }
}

Next read the 5 column VS dataset stored as a CSV file.

In [15]:
csv_5col_df = pd.read_csv('data/incremental_vs.csv', usecols=cols, dtype=col_dtypes)

Now we set the column headers for the 5 columns in the dataset and the read in the rows from the dataframe and insert them into the itemData list.

In [16]:
vs_5col_json["clinicalData"]["itemGroupData"]["IG.VS"]["items"] = [var_defn for var_defn in dataset_attrs['items'] if var_defn['name'] in cols]
vs_5col_json["clinicalData"]["itemGroupData"]["IG.VS"]["itemData"] = csv_df.values.tolist()

Show the number of data rows being written to the Dataset-JSON file.

In [17]:
print(f"writing {len(vs_5col_json['clinicalData']['itemGroupData']['IG.VS']['itemData'])} dataset rows to Dataset-JSON")

writing 1414 dataset rows to Dataset-JSON


Finally, write the 5 column VS dataset to a Dataset-JSON file

In [18]:
with open("data/vs_5col.json", "w") as fo:
    json.dump(vs_5col_json, fo)